In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv

from terrain_nerf.autonav import arc

# autoreload
%load_ext autoreload
%autoreload 2

Global map

In [ ]:
img = image.imread('../data/airsim/images/global_map.png')

In [ ]:
UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([210111.421875, 111218.84375, 32213.0])

goal_meters = (UNREAL_GOAL - UNREAL_PLAYER_START)[:2] / 100.0
goal_meters

In [ ]:
# Define a path in image coordinates
start_px = np.array([669, 157])
goal_px = np.array([52, 1469])

path = np.array([[669, 157],
                 [638, 231],
                 [622, 330],
                 [650, 414],
                 [671, 506],
                 [669, 602],
                 [651, 707],
                 [635, 812],
                 [608, 901],
                 [578, 999],
                 [531, 1107],
                 [463, 1215],
                 [387, 1249],
                 [287, 1277],
                 [186, 1310],
                 [120, 1356],
                 [79, 1420],
                 [52, 1469]])

# Plot image with plotly
# Resize image to (m,n,3)
fig = px.imshow(img[:, :, :3])
fig.add_trace(go.Scatter(x=path[:,1], y=path[:,0], mode='lines', name='path'))
fig.update_layout(width=1600, height=800)
fig.show()

In [ ]:
# Convert path to meters
path_meters = path.copy()
# Convert to float array
path_meters = path_meters.astype(np.float64)

scale_factor = 2.5  # 1 pixel = 2.5 meters (computed from goal_meters[0] / (start_px[1] - goal_px[1]))
# Shift so start is at 0
path_meters -= start_px
# Scale
path_meters *= scale_factor
# Swap x and y
path_meters = path_meters[:, [1, 0]]

path_meters

In [ ]:
# Export path to file
np.save('../data/airsim/global_path.npy', path_meters)